In [1]:
import openai
import os
from openai import cli
import pandas as pd
import urllib.request
import json 
import urllib.request
from PIL import Image
import matplotlib.pyplot as plt
import requests
import time
import os
import matplotlib.image as mpimg
import io
import azure.ai.vision as visionsdk
import base64
import concurrent.futures
from azure.storage.blob import BlobServiceClient, ContentSettings
import ast
import numpy as np
import math

In [31]:
df = pd.read_csv('Coles_products.csv', index_col=[0])
recipe = pd.read_csv('recipe.csv', index_col=[0])

In [38]:
def vectorize_image(image_url):
    # Set API endpoint and subscription key
    api_endpoint = "https://eastus.cognitiveservices.azure.com/computervision/retrieval:vectorizeImage?api-version=2023-02-01-preview"
    subscription_key = "3db11cb901f146db83293d0598176f51"

    # Set request headers
    headers = {
        "Content-Type": "application/json",
        "Ocp-Apim-Subscription-Key": subscription_key
    }

    # Set request body
    data = {
        "url": image_url
    }

    # Send POST request with headers and data
    response = requests.post(api_endpoint, headers=headers, data=json.dumps(data))

    # Parse response JSON and extract vector
    response_json = response.json()
    real_vector = response_json.get("vector")

    return real_vector

def vectorize_text(text):
    # Set API endpoint and subscription key
    api_endpoint = "https://eastus.cognitiveservices.azure.com/computervision/retrieval:vectorizeText?api-version=2023-02-01-preview"
    subscription_key = "3db11cb901f146db83293d0598176f51"

    # Set request headers
    headers = {
        "Content-Type": "application/json",
        "Ocp-Apim-Subscription-Key": subscription_key
    }

    # Set request body
    data = {
        "text": text
    }

    # Send POST request with headers and data
    response = requests.post(api_endpoint, headers=headers, data=json.dumps(data))

    # Parse response JSON and extract vector
    response_json = response.json()
    real_vector = response_json.get("vector")

    return real_vector



def get_cosine_similarity(vector1, vector2):
    dot_product = sum(vector1[i] * vector2[i] for i in range(min(len(vector1), len(vector2))))
    magnitude1 = math.sqrt(sum(x * x for x in vector1))
    magnitude2 = math.sqrt(sum(x * x for x in vector2))
    return dot_product / (magnitude1 * magnitude2)

def get_most_similar_images(real_vector, Dall_e_Vectors, image_urls):
    # Calculate cosine similarity between real_vector and each vector in Dall_e_Vectors
    similarities = [get_cosine_similarity(real_vector, vector) for vector in Dall_e_Vectors]

    # Find indices of Dall_e_Vectors with the highest similarities
    max_similarity_indices = sorted(range(len(similarities)), key=lambda i: similarities[i], reverse=True)[:1]

    # Return URLs of most similar images
    return [image_urls[i] for i in max_similarity_indices]

def get_most_similar_image(real_vector, Dall_e_Vectors, image_urls):
    # Calculate cosine similarity between real_vector and each vector in Dall_e_Vectors
    similarities = [get_cosine_similarity(real_vector, vector) for vector in Dall_e_Vectors]

    # Find index of Dall_e_Vectors with the highest similarity
    max_similarity_index = similarities.index(max(similarities))

    # Return URL of most similar image
    return image_urls[max_similarity_index]


def get_most_similar_items(real_vector, Dall_e_Vectors, product_name):
    # Calculate cosine similarity between real_vector and each vector in Dall_e_Vectors
    similarities = [get_cosine_similarity(real_vector, vector) for vector in Dall_e_Vectors]

    # Find index of Dall_e_Vectors with the highest similarity
    max_similarity_index = similarities.index(max(similarities))

    # Return URL of most similar image
    return product_name[max_similarity_index]

In [8]:
df

,product_name,product_id,price,prompt,dalle_url,final_url,vector
0,milk,1,3.49,"A carton of fresh, retail-ready milk",https://dalleproduse.blob.core.windows.net/pri...,https://retailcontosoclothing.blob.core.window...,"[-0.9423828, 0.5722656, -0.123535156, 0.491210..."
1,butter,2,4.99,A commercially packaged block of butter,https://dalleproduse.blob.core.windows.net/pri...,https://retailcontosoclothing.blob.core.window...,"[0.6298828, -0.66015625, 0.2956543, 1.2607422,..."
2,meat,3,8.99,"A neatly packed, retail-ready piece of meat",https://dalleproduse.blob.core.windows.net/pri...,https://retailcontosoclothing.blob.core.window...,"[0.08996582, 4.1171875, -3.0507812, -5.9375, 4..."
3,rice,4,19.99,"A fully sealed, retail-ready bag of rice",https://dalleproduse.blob.core.windows.net/pri...,https://retailcontosoclothing.blob.core.window...,"[-0.7895508, 1.4912109, 0.52783203, -1.1171875..."
4,eggs,5,2.49,"A carton of fresh, store-ready eggs",https://dalleproduse.blob.core.windows.net/pri...,https://retailcontosoclothing.blob.core.window...,"[-1.3232422, -1.6542969, -0.4284668, 0.9404297..."
5,juice,6,3.99,A sealed bottle of natural fruit juice for sale,https://dalleproduse.blob.core.windows.net/pri...,https://retailcontosoclothing.blob.core.window...,"[-1.4355469, 1.3740234, -0.73876953, 1.6533203..."
6,bread,7,2.49,"A freshly baked, store-ready loaf of bread",https://dalleproduse.blob.core.windows.net/pri...,https://retailcontosoclothing.blob.core.window...,"[-4.109375, 1.875, -2.8457031, 2.2285156, -0.3..."
7,mixed_fruits,8,4.49,A commercially packaged mixed fruits,https://dalleproduse.blob.core.windows.net/pri...,https://retailcontosoclothing.blob.core.window...,"[0.47558594, 2.6503906, -2.4921875, 0.7392578,..."
8,onion,9,1.99,A net bag of fresh onions ready for sale,https://dalleproduse.blob.core.windows.net/pri...,https://retailcontosoclothing.blob.core.window...,"[2.8808594, 1.0351562, -1.6367188, -0.97265625..."
9,mixed_vegetables,10,12.99,A bag of mixed vegetables containing various t...,https://dalleproduse.blob.core.windows.net/pri...,https://retailcontosoclothing.blob.core.window...,"[0.9213867, -0.5776367, 2.2832031, -1.3017578,..."


In [6]:
def process(df,recipe,query):

    dalle_url = df['final_url']

    Dall_e_Vectors = df['vector']

    # Convert the input vectors to lists of floats
    Dall_e_Vectors = [list(map(float, vec[1:-1].split(', '))) if not isinstance(vec, float) else [vec] for vec in Dall_e_Vectors]
    # text,desc = image_desc(image_url)

    real_vector = vectorize_text(query)

    real_vector = list(map(float, real_vector))

    most_similar_image_urls = get_most_similar_images(real_vector, Dall_e_Vectors, dalle_url)
    # Create a dictionary with most_similar_image_urls as keys and their corresponding indices as values
    url_indices = {url: i for i, url in enumerate(most_similar_image_urls)}

    # st.success(desc)

    # Create a boolean mask for the rows where the 'value/img_url' matches with 'most_similar_image_urls'
    mask = df['final_url'].isin(most_similar_image_urls)

    # Get the values of the 'value/productID' column for the matching rows using the boolean mask
    matching_product_ids = df.loc[mask, 'product_name'].tolist()

    # Sort the matching_product_ids based on the index values obtained from the url_indices dictionary
    matching_product_ids = sorted(matching_product_ids, key=lambda x: url_indices[df.loc[df['product_name'] == x, 'final_url'].iloc[0]])

    # Create a boolean mask for the rows where the 'value/productID' matches with 'matching_product_ids'
    mask2 = recipe['product_name'].isin(matching_product_ids)

    # Sort the matching_product dataframe based on the order of matching_product_ids
    matching_product = recipe.loc[mask2].sort_values(by=['product_name'])
    urls = matching_product['final_url']
    titles=matching_product['product_name']
    return urls,titles,matching_product

In [14]:
recipe['product_name'][0]

"['rice', 'meat', 'onion', 'garlic', 'mixed vegetables', 'eggs', 'butter', 'vegetable oil']"

In [10]:
query = 'Thai Curry'



dalle_url = df['final_url']

Dall_e_Vectors = df['vector']

# Convert the input vectors to lists of floats
Dall_e_Vectors = [list(map(float, vec[1:-1].split(', '))) if not isinstance(vec, float) else [vec] for vec in Dall_e_Vectors]
# text,desc = image_desc(image_url)

real_vector = vectorize_text(query)

real_vector = list(map(float, real_vector))

real_vector
most_similar_image_urls = get_most_similar_images(real_vector, Dall_e_Vectors, dalle_url)
# # Create a dictionary with most_similar_image_urls as keys and their corresponding indices as values
# url_indices = {url: i for i, url in enumerate(most_similar_image_urls)}

# # st.success(desc)

# # Create a boolean mask for the rows where the 'value/img_url' matches with 'most_similar_image_urls'
# mask = df['final_url'].isin(most_similar_image_urls)

# # Get the values of the 'value/productID' column for the matching rows using the boolean mask
# matching_product_ids = df.loc[mask, 'product_name'].tolist()

# # Sort the matching_product_ids based on the index values obtained from the url_indices dictionary
# matching_product_ids = sorted(matching_product_ids, key=lambda x: url_indices[df.loc[df['product_name'] == x, 'final_url'].iloc[0]])

# # Create a boolean mask for the rows where the 'value/productID' matches with 'matching_product_ids'
# mask2 = recipe['product_name'].isin(matching_product_ids)

# # Sort the matching_product dataframe based on the order of matching_product_ids
# matching_product = recipe.loc[mask2].sort_values(by=['product_name'])
# urls = matching_product['final_url']
# titles=matching_product['product_name']


In [11]:
most_similar_image_urls

['https://retailcontosoclothing.blob.core.windows.net/contoso-cg-shopping/curry_paste.jpg']

In [ ]:
import pandas as pd

# Assuming df and recipe are your dataframes
df.set_index('product_name', inplace=True)

def get_urls(product_list):
    urls = []
    for product in product_list:
        if product in df.index:
            urls.append(df.loc[product, 'final_url'])
        else:
            urls.append(None)  # or '' or any value indicating missing url
    return urls

recipe['final_url'] = recipe['product_name'].apply(get_urls)


In [32]:
import ast

# let's say the query string is "Fried Rice with Meat and Vegetables"
query = "Fried Rice with Meat and Vegetables"

text=vectorize_text(query)

most_similar_image_urls = get_most_similar_images(text, Dall_e_Vectors, dalle_url)

# find the match in the recipe dataframe
matched_recipe = recipe[recipe['food'] == query]

# check if we have a match
if not matched_recipe.empty:
    # get the product names
    product_names_str = matched_recipe['product_name'].values[0]
    # Convert the string representation of list to actual list
    product_names = ast.literal_eval(product_names_str)
    
    # replace spaces with underscores
    product_names = [name.replace(' ', '_') for name in product_names]

    # find the final_url and product_name from df
    matched_df = df[df['product_name'].isin(product_names)][['product_name', 'final_url']]
else:
    print("No matching food found for the query.")


In [33]:
matched_df

,product_name,final_url
1,butter,https://retailcontosoclothing.blob.core.window...
2,meat,https://retailcontosoclothing.blob.core.window...
3,rice,https://retailcontosoclothing.blob.core.window...
4,eggs,https://retailcontosoclothing.blob.core.window...
8,onion,https://retailcontosoclothing.blob.core.window...
9,mixed_vegetables,https://retailcontosoclothing.blob.core.window...
10,vegetable_oil,https://retailcontosoclothing.blob.core.window...


In [34]:
recipe

,food,product_name,vector
0,Fried Rice with Meat and Vegetables,"['rice', 'meat', 'onion', 'garlic', 'mixed veg...","[-0.009019881, -0.0013037574, 0.031162385, 0.0..."
1,Thai Curry,"['vegetable oil', 'curry paste', 'coconut milk...","[-0.024356565, 0.030535305, 0.006356006, -0.03..."
2,One-Pot Cheesy Chicken and Rice,"['chicken', 'onion', 'rice', 'chicken broth', ...","[0.008676969, 0.009790083, 0.0017223926, 0.010..."


In [36]:
text=vectorize_text(query)
Dall_e_Vectors = recipe['vector']
Dall_e_Vectors = [list(map(float, vec[1:-1].split(', '))) if not isinstance(vec, float) else [vec] for vec in Dall_e_Vectors]
product_name=recipe['product_name']
most_similar_items = get_most_similar_items(text, Dall_e_Vectors, product_name)


In [37]:
most_similar_image_urls

["['rice', 'meat', 'onion', 'garlic', 'mixed vegetables', 'eggs', 'butter', 'vegetable oil']"]

In [1]:
from st_clickable_images import clickable_images
import streamlit as st
from PIL import Image
import json
import io
import tempfile
import requests
import math
import pandas as pd
import azure.ai.vision as visionsdk
from azure.storage.blob import BlobServiceClient
import urllib.parse
@st.cache_data
def load_model():

    df = pd.read_csv('./toy_data.csv',index_col=[0])
    return df

def vectorize_image(image_url):
    # Set API endpoint and subscription key
    api_endpoint = "https://eastus.cognitiveservices.azure.com/computervision/retrieval:vectorizeImage?api-version=2023-02-01-preview"
    subscription_key = "3db11cb901f146db83293d0598176f51"

    # Set request headers
    headers = {
        "Content-Type": "application/json",
        "Ocp-Apim-Subscription-Key": subscription_key
    }

    # Set request body
    data = {
        "url": image_url
    }

    # Send POST request with headers and data
    response = requests.post(api_endpoint, headers=headers, data=json.dumps(data))

    # Parse response JSON and extract vector
    response_json = response.json()
    real_vector = response_json.get("vector")

    return real_vector


def vectorize_images(image_urls,text):
    # Set API endpoint and subscription key
    api_endpoint = "https://eastus.cognitiveservices.azure.com/computervision/retrieval:vectorizeImage?api-version=2023-02-01-preview"
    subscription_key = "3db11cb901f146db83293d0598176f51"

    # Set request headers
    headers = {
        "Content-Type": "application/json",
        "Ocp-Apim-Subscription-Key": subscription_key
    }

    # Initialize empty list for results
    Dall_e_Vectors = []

    # Vectorize each image URL and store result in list
    for i, (image_url, text) in enumerate(zip(image_urls, texts)):
        # Set request body
        data = {
            "url": image_url,
            "text": text
        }

        # Send POST request with headers and data
        response = requests.post(api_endpoint, headers=headers, data=json.dumps(data))

        # Parse response JSON and extract vector
        response_json = response.json()
        real_vector = response_json.get("vector")

        # Append vector to list
        Dall_e_Vectors.append(real_vector)
        
        print(f"Processed image {i+1} of {len(image_urls)}")

    return Dall_e_Vectors

def vectorize_text(text):
    # Set API endpoint and subscription key
    api_endpoint = "https://eastus.cognitiveservices.azure.com/computervision/retrieval:vectorizeText?api-version=2023-02-01-preview"
    subscription_key = "3db11cb901f146db83293d0598176f51"

    # Set request headers
    headers = {
        "Content-Type": "application/json",
        "Ocp-Apim-Subscription-Key": subscription_key
    }

    # Set request body
    data = {
        "text": text
    }

    # Send POST request with headers and data
    response = requests.post(api_endpoint, headers=headers, data=json.dumps(data))

    # Parse response JSON and extract vector
    response_json = response.json()
    real_vector = response_json.get("vector")

    return real_vector

def vectorize_texts(texts):
    # Set API endpoint and subscription key
    api_endpoint = "https://eastus.cognitiveservices.azure.com/computervision/retrieval:vectorizeText?api-version=2023-02-01-preview"
    subscription_key = "3db11cb901f146db83293d0598176f51"

    # Set request headers
    headers = {
        "Content-Type": "application/json",
        "Ocp-Apim-Subscription-Key": subscription_key
    }
    Dall_e_Vectors = []

    # Vectorize each image URL and store result in list
    for i, text in enumerate(texts):
        # Set request body
        data = {
            "text": text
        }

        # Send POST request with headers and data
        response = requests.post(api_endpoint, headers=headers, data=json.dumps(data))

        # Parse response JSON and extract vector
        response_json = response.json()
        real_vector = response_json.get("vector")

        # Append vector to list
        Dall_e_Vectors.append(real_vector)
        
        print(f"Processed image {i+1} of {len(texts)}")

    return Dall_e_Vectors

def get_cosine_similarity(vector1, vector2):
    dot_product = sum(vector1[i] * vector2[i] for i in range(min(len(vector1), len(vector2))))
    magnitude1 = math.sqrt(sum(x * x for x in vector1))
    magnitude2 = math.sqrt(sum(x * x for x in vector2))
    return dot_product / (magnitude1 * magnitude2)

def get_most_similar_images(real_vector, Dall_e_Vectors, image_urls):
    # Calculate cosine similarity between real_vector and each vector in Dall_e_Vectors
    similarities = [get_cosine_similarity(real_vector, vector) for vector in Dall_e_Vectors]

    # Find indices of Dall_e_Vectors with the highest similarities
    max_similarity_indices = sorted(range(len(similarities)), key=lambda i: similarities[i], reverse=True)[:15]

    # Return URLs of most similar images
    return [image_urls[i] for i in max_similarity_indices]

def get_most_similar_image(real_vector, Dall_e_Vectors, image_urls):
    # Calculate cosine similarity between real_vector and each vector in Dall_e_Vectors
    similarities = [get_cosine_similarity(real_vector, vector) for vector in Dall_e_Vectors]

    # Find index of Dall_e_Vectors with the highest similarity
    max_similarity_index = similarities.index(max(similarities))

    # Return URL of most similar image
    return image_urls[max_similarity_index]

def get_most_similar_items(real_vector, Dall_e_Vectors, product_name):
    # Calculate cosine similarity between real_vector and each vector in Dall_e_Vectors
    similarities = [get_cosine_similarity(real_vector, vector) for vector in Dall_e_Vectors]

    # Find index of Dall_e_Vectors with the highest similarity
    max_similarity_index = similarities.index(max(similarities))

    # Return URL of most similar image
    return product_name[max_similarity_index]

def process(df,if_url,query):

    image_url = df['image_path']

    Vectors = df['vector']

    # Convert the input vectors to lists of floats
    Vectors = [list(map(float, vec[1:-1].split(', '))) if not isinstance(vec, float) else [vec] for vec in Vectors]
    # text,desc = image_desc(image_url)

    real_vector = vectorize_text(query)

    real_vector = list(map(float, real_vector))

    most_similar_image_urls = get_most_similar_images(real_vector, Vectors, image_url)
    # Create a dictionary with most_similar_image_urls as keys and their corresponding indices as values
    url_indices = {url: i for i, url in enumerate(most_similar_image_urls)}

    # st.success(desc)

    # Create a boolean mask for the rows where the 'value/img_url' matches with 'most_similar_image_urls'
    matching_product = df['image_path'].isin(most_similar_image_urls)

    urls = matching_product['image_path']
    titles=matching_product['product_name']
    return urls,titles,matching_product


def tempfile(query):
    import tempfile
    response = requests.get(query)
    with tempfile.NamedTemporaryFile(suffix=".jpg", delete=False) as temp_file:
        temp_file.write(response.content)
        image_path = temp_file.name
    img = Image.open(image_path)
    new_image = img.resize((300, 400))
    return new_image

2023-06-16 10:42:17.083 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager


In [3]:

from st_clickable_images import clickable_images
import streamlit as st
from PIL import Image
import json
import io
import tempfile
import requests
import math
import pandas as pd
import azure.ai.vision as visionsdk
from azure.storage.blob import BlobServiceClient
import urllib.parse
import numpy as np
import math
import base64

import ast


df= load_model()

query="robot"
# Parse the input text as a URL
parsed_url = urllib.parse.urlparse(query)

# Check if the scheme component of the URL is not empty, indicating that it is a valid URL
if parsed_url.scheme != "":
    st.write("Input query is a URL")
    if_url='True'
else:
    st.write("Input query is text.")
    print("Input query is text")
    if_url='False'

if query and if_url == 'True':
# Save the image from URL to a temporary directory
    new_image=tempfile(query)
    st.sidebar.image(new_image, caption="Input Image")
if len(query) > 0:
    urls,titles,matching_product=process(df,if_url,query)

    clicked = clickable_images(
        [url for url in urls],
        titles=[title for title in titles],
        div_style={
            "display": "flex",
            "justify-content": "center",
            "flex-wrap": "wrap",
        },
        img_style={"margin": "2px", "height": "200px"},
    )
    if clicked >= 0:
        change_query = False
        if "last_clicked" not in st.session_state:
            change_query = True
        else:
            if clicked != st.session_state["last_clicked"]:
                change_query = True
        if change_query:
            url = matching_product.iloc[clicked]['image_path']
            st.session_state["query"] = url
            st.experimental_rerun()


2023-06-16 10:43:04.512 
  command:

    streamlit run C:\Users\shado\AppData\Roaming\Python\Python310\site-packages\ipykernel_launcher.py [ARGUMENTS]
2023-06-16 10:43:04.514 No runtime found, using MemoryCacheStorageManager


Input query is text


KeyError: 'image_path'

In [16]:
image_url = df['image_path']
Vectors = df['vector']

# Convert the input vectors to lists of floats
Vectors = [list(map(float, vec[1:-1].split(', '))) if not isinstance(vec, float) else [vec] for vec in Vectors]

# assume that vectorize_text and get_most_similar_images are defined elsewhere
real_vector = vectorize_text(query)
real_vector = list(map(float, real_vector))

most_similar_image_urls = get_most_similar_images(real_vector, Vectors, image_url)

# Create a boolean mask for the rows where the 'image_path' matches with 'most_similar_image_urls'
matching_product = df[df['image_path'].isin(most_similar_image_urls)]

# Order the dataframe according to most_similar_image_urls
matching_product = matching_product.set_index('image_path').loc[most_similar_image_urls].reset_index()

urls = matching_product['image_path']
titles = matching_product['product_name']


In [21]:
images = []
for file in urls:
    with open(file, "rb") as image:
        encoded = base64.b64encode(image.read()).decode()
        images.append(f"data:image/jpeg;base64,{encoded}")

In [23]:
clicked = clickable_images(
    [url for url in urls],
    titles=[title for title in titles],
    div_style={
        "display": "flex",
        "justify-content": "center",
        "flex-wrap": "wrap",
    },
    img_style={"margin": "2px", "height": "200px"},
        )

In [24]:
clicked

-1

In [13]:
df['image_path'][0]

'G:\\VSCode\\Personal_work\\Vectorized_Search\\image\\5\\Sparkling Sea Mermaid Doll.jpeg'

In [28]:
df = pd.read_csv('./toy_data.csv',index_col=[0])

In [29]:
df

,product_name,product_id,price,prompts,dalle_url,vector,image_path,url
0,Sparkling Sea Mermaid Doll,1,24.99,"Sparkling Sea Mermaid Doll, kid's toy",https://dalleproduse.blob.core.windows.net/pri...,"[3.7792969, 3.7695312, 0.97753906, -1.7988281,...",G:\VSCode\Personal_work\Vectorized_Search\app\...,https://www.flickr.com/photos/198541622@N02/52...
1,Mighty Monster Trucks Set,2,39.99,"Mighty Monster Trucks Set, kid's toy",https://dalleproduse.blob.core.windows.net/pri...,"[-2.0078125, -0.13623047, 0.14624023, 0.624023...",G:\VSCode\Personal_work\Vectorized_Search\app\...,https://www.flickr.com/photos/198541622@N02/52...
2,Playful Pet Vet Clinic,3,34.99,"Playful Pet Vet Clinic, kid's toy",https://dalleproduse.blob.core.windows.net/pri...,"[2.1542969, -2.9160156, -0.17004395, -1.238281...",G:\VSCode\Personal_work\Vectorized_Search\app\...,https://www.flickr.com/photos/198541622@N02/52...
3,Rainbow Roller Skates,4,29.99,"Rainbow Roller Skates, kid's toy",https://dalleproduse.blob.core.windows.net/pri...,"[1.5087891, 2.9960938, -2.234375, 1.5068359, 3...",G:\VSCode\Personal_work\Vectorized_Search\app\...,https://www.flickr.com/photos/198541622@N02/52...
4,Build-A-Bot Robot Kit,5,54.99,"Build-A-Bot Robot Kit, kid's toy",https://dalleproduse.blob.core.windows.net/pri...,"[1.9951172, 1.3916016, -3.0, 1.1318359, 0.2440...",G:\VSCode\Personal_work\Vectorized_Search\app\...,https://www.flickr.com/photos/198541622@N02/52...
...,...,...,...,...,...,...,...,...
82,Classic Pick-Up Sticks Game,95,12.99,"Classic Pick-Up Sticks Game, kid's toy",https://dalleproduse.blob.core.windows.net/pri...,"[1.2539062, -0.71240234, 0.062805176, 1.160156...",G:\VSCode\Personal_work\Vectorized_Search\app\...,https://www.flickr.com/photos/198541622@N02/52...
83,Miniature Tennis Set,97,19.99,"Miniature Tennis Set, kid's toy",https://dalleproduse.blob.core.windows.net/pri...,"[0.41577148, -1.9677734, -0.8305664, -1.500976...",G:\VSCode\Personal_work\Vectorized_Search\app\...,https://www.flickr.com/photos/198541622@N02/52...
84,My First Sewing Kit,98,24.99,"My First Sewing Kit, kid's toy",https://dalleproduse.blob.core.windows.net/pri...,"[-4.7539062, 0.6020508, 1.8261719, 2.1640625, ...",G:\VSCode\Personal_work\Vectorized_Search\app\...,https://www.flickr.com/photos/198541622@N02/52...
85,Foam Boomerang Set,99,16.99,"Foam Boomerang Set, kid's toy",https://dalleproduse.blob.core.windows.net/pri...,"[1.0820312, -0.16967773, 0.89746094, -0.216308...",G:\VSCode\Personal_work\Vectorized_Search\app\...,https://www.flickr.com/photos/198541622@N02/52...


In [31]:
df['url'][0]

'https://www.flickr.com/photos/198541622@N02/52978692972/'

In [34]:
from IPython.display import display, Image
display(Image(url='https://www.seiu1000.org/sites/main/files/main-images/camera_lense_0.jpeg'))
